<a href="https://colab.research.google.com/github/Jiho-korea/Starcraft2MatchPredictionAlgorithm/blob/master/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=4ac3c3228d64dc9163b742196e95ae4170f542856e7b358dbba05a82950c556c
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
!pip install catboost

     |████████████████████████████████| 66.3MB 99kB/s 


In [ ]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import re
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV 
from sklearn.model_selection import StratifiedKFold, cross_val_score
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시

import itertools

from sklearn.preprocessing import LabelEncoder

from xgboost import plot_importance
from xgboost import XGBClassifier

from catboost import CatBoostClassifier

In [ ]:
from google.colab import drive
drive.mount('/gdrive/', force_remount=True)

Mounted at /gdrive/


In [ ]:
!ls "/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset"

sample_submission.csv  test.csv  train.csv


In [ ]:
train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/train.csv')

In [ ]:
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/test.csv')

In [ ]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [ ]:
def get_ablility_code(x): # ability_code 계산
    try:
        return x.split('-')[0].strip('( )')
    except:
        return np.nan

In [ ]:
def get_camera_pos(x, pos):
        try:
            return float(x.strip('at ( )').split(',')[pos])
        except BaseException as e:
            return np.nan

In [ ]:
train.loc[train['event']=='Ability','ability_code'] = train.loc[train['event']=='Ability','event_contents'].apply(lambda x: get_ablility_code(x))
test.loc[test['event']=='Ability','ability_code'] = test.loc[test['event']=='Ability','event_contents'].apply(lambda x: get_ablility_code(x))

In [ ]:
train.loc[train['event']=='Camera','camera_pos_x'] = train.loc[train['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,0))
train.loc[train['event']=='Camera','camera_pos_y'] = train.loc[train['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,1))
test.loc[test['event']=='Camera','camera_pos_x'] = test.loc[test['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,0))
test.loc[test['event']=='Camera','camera_pos_y'] = test.loc[test['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,1))

In [ ]:
train.head()

,game_id,winner,time,player,species,event,event_contents,ability_code,camera_pos_x,camera_pos_y
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",NaN,145.250000,21.507812
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)",NaN,22.750000,147.007812
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]'],NaN,NaN,NaN
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV,1360,NaN,NaN
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",NaN,142.996094,24.503906


In [ ]:
unit_mineral={
    '15E0' : 50, # TrainProbe
    '1586' : 100,   #
    '15A8' : 150,  #
    '15A0' : 150,  #
    '1581' : 125,  #
    '15C0' : 200,  #
    '15C1' : 25,  #
    '1580' : 100,  #
    '15A4' : 250,  #
    '15C3' : 275,  #
    '1585' : 50,  #
    '15A2' : 350,  #
    '15C2' : 300,  #
    '15D2' : 150,  #
    '1583' : 50,   # 
    '15A9' : 250, #
    '2720' : 400, #
    '1360' : 50, #
    '13E0' : 50, #
    '1420' : 100, #
    '13E1' : 50, #
    '13E3' : 150, #
    '1421' : 150, #
    '1407' : 150, #
    '1426' : 150, #
    '1422' : 100, #
    '1424' : 150, #
    '13E2' : 100, #
    '1423' : 400, #
    '1401' : 150,  #
    '1404' : 300,  #
    '1418' : 75,  #
    '1820' : 50,  #
    '1822' : 100,  #
    '1821' : 50,    # 
    '1823' : 100,   #
    '1829' : 75,  #
    '1BA0' : 50,  #
    '4020' : 25,  #
    '1824' : 100,   #
    '182B' : 150,  #
    '182E' : 100,  #
    '1840' : 150,  #
    '182A' : 100,   #
    '1E60' : 150,  #
    '920'  : 25   #
}

In [ ]:
unit_gas = {
    '1586'  :  25 ,    #- TrainAdept/지상
    '15A8'  :  150,    #- TrainOracle/x(마법)
    '15A0'  :  100,    #- TrainPhoenix/공중
    '1581'  :  50 ,    #- TrainStalker/지상
    '15C1'  :  75 ,    #- TrainObserver/x
    '15A4'  :  150,    #- TrainVoidRay/지상,공중
    '15C3'  :  100,    #- TrainImmortal/지상
    '1585'  :  100,    #- TrainSentry/지상,공중 (마법)
    '15A2'  :  250,    #- TrainCarrier/지상,공중
    '15C2'  :  200,    #- TrainColossus/지상
    '15D2'  :  150,    #- TrainDisruptor/x(마법)
    '1583'  :  150,    #- TrainHighTemplar/x(마법)
    '15A9'  :  200,    #- TrainTempest/공중
    '2720'  :  100,    #- TrainMother/지상
    '1420'  :  100,    #- TrainMedivac/x
    '13E1'  :  50 ,    #- TrainReaper/지상
    '13E3'  :  25 ,    #- TrainMarauder/x
    '1421'  :  100,    #- TrainBanshee/지상
    '1407'  :  100,    #- TrainCyclone/지상,공중
    '1426'  :  100,    #- TrainLiberator/공중,지상
    '1422'  :  200,    #- TrainRaven/마법*
    '1424'  :  75 ,    #- TrainViking/지상,공중
    '13E2'  :  50 ,    #- TrainGhost/지상,공중
    '1423'  :  300,    #- TrainBattlecruiser/지상,공중
    '1401'  :  125,    #- Crucio지상
    '1404'  :  200,    #- Thor지상공중
    '1823'  :  50 ,    #- MorphHydralisk/지상,공중
    '1829'  :  25 ,    #- MorphRoach/지상
    '1BA0'  :  50,    # MorphToOverseer/x
    '4020'  :  75 ,    #- MorphToRavager/지상
    '1824'  :  100,    #- MorphMutalisk/지상,공중
    '182B'  :  100,    #- MorphCorruptor/공중
    '182E'  :  75 ,    #- MorphSwarmHost/지상
    '1840'  :  150,    #- MorphToBroodLord/지상
    '182A'  :  150,    #- MorphInfestor/x(마법)
    '920' : 25 #- Baneling지상   
}

In [ ]:
# 건물 코드 리스트
building_0 = ['0_1541','0_1021','0_1543','0_1023','0_1541','0_1547','0_102A','0_1260','0_1261','0_102B','0_1024','0_1025','0_154E','0_12A0','0_154D','0_16E4','0_1549','0_16EE','0_1546','0_1020','0_16EF','0_12A1','0_12E1','0_16EA','0_16ED','0_1026','0_102D','0_16E3','0_12E0','0_16E5','0_16E0','0_16E6','0_154B','0_154C','0_154A','0_16E8','0_1542','0_16E2','0_1545','0_1028','0_1022','0_1540','0_102F','0_1029','0_16E9','0_2180','0_16E7']
building_1 = ['1_1541','1_1021','1_1543','1_1023','1_1541','1_1547','1_102A','1_1260','1_1261','1_102B','1_1024','1_1025','1_154E','1_12A0','1_154D','1_16E4','1_1549','1_16EE','1_1546','1_1020','1_16EF','1_12A1','1_12E1','1_16EA','1_16ED','1_1026','1_102D','1_16E3','1_12E0','1_16E5','1_16E0','1_16E6','1_154B','1_154C','1_154A','1_16E8','1_1542','1_16E2','1_1545','1_1028','1_1022','1_1540','1_102F','1_1029','1_16E9','1_2180','1_16E7']

In [ ]:
building_mineral ={
    '1541'  :  100,   #   BuildPylon                
    '1021'  :  100,   #   BuildSupplyDepot       
    '1543'  :  150,   #   BuildGateway           
    '1023'  :  150,   #   BuildBarracks          
    '1547'  :  150,   #   BuildPhotonCannon      
    '102A'  :  200,   #  BuildFactory           
    '1260'  :   50,   #   BuildBarracksTechLab   
    '1261'  :  50,   #    BuildBarracksReactor   
    '102B'  :  150,   #    BuildStarport         
    '1024'  :  125,   #   BuildEngineeringBay    
    '1025'  :  100,   #   BuildMissileTurret     
    '154E'  :  150,   #    BuildCyberneticsCore   
    '12A0'  :  50,   #    BuildFactoryTechLab    
    '154D'  :  200,   #    BuildRoboticsFacility 
    '1544'  :  150,   #   BuildForge             
    '16E4'  :   75,   #  BuildEvolutionChamber  
    '1549'  :  150,   #    BuildStargate         
    '16EE'  :  100,   #   BuildSpineCrawler      
    '1546'  :  150,   #    BuildTwilightCouncil  
    '1020'  :  400,   #   BuildCommandCenter     
    '16EF'  :  100,   #   BuildSporeCrawler      
    '12A1'  :  50,   #    BuildFactoryReactor    
    '12E1'  :  50,   #    BuildStarportReactor   
    '16EA'  :  100,   #    BuildBanelingNest      
    '16ED'  :  150,   #  BuildRoachWarren       
    '1026'  :  100,   #   BuildBunker            
    '102D'  :  150,   #    BuildArmory           
    '16E3'  :  200,   #   BuildSpawningPool         
    '12E0'  :  50,   #    BuildStarportTechLab   
    '16E5'  :  100,   #    BuildHydraliskDen     
    '16E0'  :  400,   #    BuildHatchery            
    '16E6'  :  200,   #    BuildSpire            
    '154B'  :  100,   #     BuildDarkShrine      
    '154C'  :  150,   #     BuildRoboticsBay     
    '154A'  :  150,   #     BuildTemplarArchive  
    '16E8'  :  100,   #    BuildInfestationPit   
    '1542'  :  100,   #   BuildAssimilator          
    '16E2'  :  75,   #  BuildExtractor            
    '1545'  :  300,   #    BuildFleetBeacon      
    '1028'  :  100,   #    BuildSensorTower       
    '1022'  :  75,   #  BuildRefinery             
    '1540'  :  400,   #   BuildNexus                
    '102F'  :  200,   #    BuildFusionCore       
    '1029'  :  150,   #    BuildGhostAcademy      
    '16E9'  :  150,   #    BuildNydusNetwork     
    '2180'  :  150,   #   BuildNydusCanal           
    '16E7'  :  150   #    BuildUltraliskCavern  
}

In [ ]:
building_gas ={  
    '102A' : 50   ,   #  BuildFactory           
    '1260' : 50   ,   #   BuildBarracksTechLab   
    '1261' : 50    ,  #    BuildBarracksReactor   
    '102B' : 100  ,   #    BuildStarport         
    '12A0' : 50   ,  #    BuildFactoryTechLab    
    '154D' : 20  ,   #    BuildRoboticsFacility 
    '1549' : 15  ,   #    BuildStargate            
    '1546' : 100  ,   #    BuildTwilightCouncil    
    '12A1' : 50   ,  #    BuildFactoryReactor    
    '12E1' : 50   ,  #    BuildStarportReactor   
    '16EA' : 50   ,   #    BuildBanelingNest            
    '102D' : 100  ,   #    BuildArmory               
    '12E0' : 50   ,  #    BuildStarportTechLab   
    '16E5' : 100  ,   #    BuildHydraliskDen            
    '16E6' : 200  ,   #    BuildSpire            
    '154B' : 25  ,   #     BuildDarkShrine      
    '154C' : 15  ,   #     BuildRoboticsBay     
    '154A' : 20  ,   #     BuildTemplarArchive  
    '16E8' : 100  ,   #    BuildInfestationPit         
    '1545' : 200  ,   #    BuildFleetBeacon      
    '1028' : 50   ,   #    BuildSensorTower                 
    '102F' : 200  ,   #    BuildFusionCore       
    '1029' : 50   ,   #    BuildGhostAcademy     
    '16E9' : 200  ,   #    BuildNydusNetwork            
    '16E7' : 200       #    BuildUltraliskCavern  
}

In [ ]:
# 일꾼 코드 리스트 추가
worker_0 = ['0_1360', '0_15E0', '0_1820']
worker_1 = ['1_1360', '1_15E0', '1_1820']
worker = worker_0 + worker_1

# 일꾼 선택 정보 리스트
target_worker_0 = ['0_Target_SCV','0_Target_Probe','0_Target_Drone']
target_worker_1 = ['1_Target_SCV','1_Target_Probe','1_Target_Drone']

target_worker=target_worker_0 + target_worker_1

# 일꾼 공격 정보 리스트
attack_worker_0 = ['0_Attack_SCV', '0_Attack_Probe', '0_Attack_Drone']
attack_worker_1 = ['1_Attack_SCV', '1_Attack_Probe', '1_Attack_Drone']
attack_worker = attack_worker_0 + attack_worker_1



# 유닛 코드 리스트(프, 테, 저) 추가 

# 프로토스 유닛 코드
              #  Adept  Oracle  Phoenix  Stalker  WarpPrism  Observer  Zealot  VoidRay  Immortal  Sentry  Carrier  Colossus  Disruptor  HighTemplar  Tempest  MothershipCore
unit_code_P_0 = ['0_1586', '0_15A8', '0_15A0', '0_1581', '0_15C0', '0_15C1', '0_1580', '0_15A4', '0_15C3', '0_1585', '0_15A2', '0_15C2', '0_15D2', '0_1583', '0_15A9', '0_2720']
unit_code_P_1 = ['1_1586', '1_15A8', '1_15A0', '1_1581', '1_15C0', '1_15C1', '1_1580', '1_15A4', '1_15C3', '1_1585', '1_15A2', '1_15C2', '1_15D2', '1_1583', '1_15A9', '1_2720']          

# 테란 유닛 코드 
              #  Marine  Medivac  Reaper  Marauder  Banshee  Cyclone  Liberator  Raven  Viking  Ghost  Battlecruiser  Crucio  Thor  Mine
unit_code_T_0 = ['0_13E0', '0_1420', '0_13E1', '0_13E3', '0_1421', '0_1407', '0_1426', '0_1422', '0_1424', '0_13E2', '0_1423', '0_1401', '0_1404', '0_1418']
unit_code_T_1 = ['1_13E0', '1_1420', '1_13E1', '1_13E3', '1_1421', '1_1407', '1_1426', '1_1422', '1_1424', '1_13E2', '1_1423', '1_1401', '1_1404', '1_1418']

# 저그 유닛 코드
              #  Overlord  Zergling  Hydralisk  Roach  ToOverseer  ToRavager  Mutalisk  Corruptor  SwarmHost  ToBroodLord  Infestor  Baneling 
unit_code_Z_0 = ['0_1822', '0_1821', '0_1823', '0_1829', '0_1BA0', '0_4020', '0_1824', '0_182B', '0_182E', '0_1840', '0_182A', '0_1E60', '0_920']
unit_code_Z_1 = ['1_1822', '1_1821', '1_1823', '1_1829', '1_1BA0', '1_4020', '1_1824', '1_182B', '1_182E', '1_1840', '1_182A', '1_1E60', '1_920']

# ground    # Stalker Zealot Immortal Sentry Colossus Disruptor HighTemplar Marine Reaper Marauder Crucio Ghost Thor Viking Cyclone Zergling Beneling Queen Hydralisk Roach Infestor SwarmHost
unit_ground_0=['0_1581', '0_1580', '0_15C3', '0_1585', '0_15C2', '0_15D2', '0_1583', '0_13E0', '0_13E1', '0_13E3', '0_1401', '0_13E2', '0_1404', '0_1424', '0_1407', '0_1821', '0_920', '0_1E60', '0_1823', '0_1829', '0_182A', '0_182E']
unit_ground_1=['1_1581', '1_1580', '1_15C3', '1_1585', '1_15C2', '1_15D2', '1_1583', '1_13E0', '1_13E1', '1_13E3', '1_1401', '1_13E2', '1_1404', '1_1424', '1_1407', '1_1821', '1_920', '1_1E60', '1_1823', '1_1829', '1_182A', '1_182E']

# sky       # Oracle Phoenix WarpPrism VoidRay Carrier Tempest Mother Banshee Raven Viking Battlecruiser Medivac Mutalisk Corruptor ToBroodLord
unit_sky_0 = ['0_15A8', '0_15A0', '0_15C0', '0_15A4', '0_15A2', '0_15A9', '0_2720',  '0_1421', '0_1422', '0_1424', '0_1423', '0_1420', '0_1824', '0_182B', '0_1840']
unit_sky_1 = ['1_15A8', '1_15A0', '1_15C0', '1_15A4', '1_15A2', '1_15A9', '1_2720',  '1_1421', '1_1422', '1_1424', '1_1423', '1_1420', '1_1824', '1_182B', '1_1840']

# can ground atk       #Stalker Zealot VoidRay Immortal Sentry Carrier Colossus HighTemplar Tempest Mother Marine Reaper Marauder Crucio Ghost Thor Banshee Viking Battlecruiser Cyclone Zergling Beneling Queen Hydralisk Roach Mutalisk ToBroodLord SwarmHost
unit_can_atk_ground_0 = ['0_1581', '0_1580', '0_15A4', '0_15C3', '0_1585', '0_15A2', '0_15C2', '0_1583', '0_15A9', '0_2720',  '0_13E0', '0_13E1', '0_13E3', '0_1401', '0_13E2', '0_1404', '0_1421', '0_1424', '0_1423', '0_1407',   '0_1821', '0_920', '0_1E60', '0_1823', '0_1829', '0_1824', '0_1840', '0_182E']
unit_can_atk_ground_1 = ['1_1581', '1_1580', '1_15A4', '1_15C3', '1_1585', '1_15A2', '1_15C2', '1_1583', '1_15A9', '1_2720',  '1_13E0', '1_13E1', '1_13E3', '1_1401', '1_13E2', '1_1404', '1_1421', '1_1424', '1_1423', '1_1407',   '1_1821', '1_920', '1_1E60', '1_1823', '1_1829', '1_1824', '1_1840', '1_182E']

# can sky atk     # TrainPhoenix TrainStalker TrainVoidRay TrainSentry TrainCarrier TrainHighTemplar TrainTempest TrainMarine TrainGhost Thor TrainViking TrainBattlecruiser TrainCyclone Queen MorphHydralisk MorphMutalisk MorphCorruptor MorphToBroodLord
unit_can_atk_sky_0 = ['0_15A0', '0_1581', '0_15A4', '0_1585', '0_15A2', '0_1583', '0_15A9', '0_13E0', '0_13E2', '0_1404', '0_1424', '0_1423', '0_1407', '0_1E60', '0_1823', '0_1824', '0_182B', '0_1840']
unit_can_atk_sky_1 = ['1_15A0', '1_1581', '1_15A4', '1_1585', '1_15A2', '1_1583', '1_15A9', '1_13E0', '1_13E2', '1_1404', '1_1424', '1_1423', '1_1407', '1_1E60', '1_1823', '1_1824', '1_182B', '1_1840']

# cannot ground atk
unit_cannot_atk_ground_0 = ['0_15A0', '0_15C2', '0_15A9', '0_1586', '0_13E1', '0_13E1', '0_13E1', '0_182B' ]
unit_cannot_atk_ground_1 = ['1_15A0', '1_15C2', '1_15A9', '1_1586', '1_13E1', '1_13E1', '1_13E1', '1_182B' ]

# cannot sky atk
unit_cannot_atk_sky_0 = ['0_1586', '0_1581', '0_1580', '0_15C3', '0_2720', '0_13E1', '0_1421', '0_1401', '0_1821', '0_1829', '0_4020', '0_182E', '0_1840', '0_920']
unit_cannot_atk_sky_1 = ['1_1586', '1_1581', '1_1580', '1_15C3', '1_2720', '1_13E1', '1_1421', '1_1401', '1_1821', '1_1829', '1_4020', '1_182E', '1_1840', '1_920']

total_unit_code = unit_code_P_0 + unit_code_P_1 + unit_code_T_0 + unit_code_T_1 + unit_code_Z_0 + unit_code_Z_1

In [ ]:
events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']

In [ ]:
def merge_data(source, df):
    """
    merge_data함수에 넘겨지는 source는 game_id, player column을 가져야 합니다.
    각 항목을 player_0, player_1로 dataframe에 merge 시키는 역할
    
    source data
    game_id player ability_count
    1       0      10
    1       1      20
    
    merge되었을 때 df 값
    game_id player_0 player_1 ability_count_player_0 ability_count_player_1
    0       0        1        10                     20
    """
    new_c = [c for c in source.columns if c not in ['game_id','player']]
    
    source.columns = ['game_id', 'player_0'] + [f'{c}_player_0'for c in new_c]
    df = df.merge(source, on=['game_id','player_0'], how='left')
    
    source.columns = ['game_id', 'player_1'] + [f'{c}_player_1'for c in new_c]
    df = df.merge(source, on=['game_id','player_1'], how='left')
    return df

In [ ]:
def need_swap(row):
    """
    player swap이 필요한지 확인 (저테프 를 프테저로)
    T vs P -> P vs T
    Z vs P -> P vs Z
    Z vs T -> T vs Z
    """

    if row['species_0'] == 'T' and row['species_1'] == 'P':
        return 1
    elif row['species_0'] == 'Z' and row['species_1'] == 'P':
        return 1
    elif row['species_0'] == 'Z' and row['species_1'] == 'T':
        return 1
    else:
        return 0

In [ ]:
def winner_swap(org_winner):
    if org_winner == 0:
        return 1
    elif org_winner == 1:
        return 0
    else:
        raise ValueError('Incorrect Number')

In [ ]:
def swap(train, unit_code_unique, target_unique, selection_unique, attack_unique, answer=False):
    events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']

    swaped_index = train.loc[train.swap == 1].index.values

    temp_species = train.loc[train.swap == 1, 'species_0']
    train.loc[train.swap == 1, 'species_0'] = train.loc[train.swap == 1, 'species_1']
    train.loc[train.swap == 1, 'species_1'] = temp_species

    temp_event_count = train.loc[train.swap == 1, 'event_count_0']
    train.loc[train.swap == 1, 'event_count_0'] = train.loc[train.swap == 1, 'event_count_1']
    train.loc[train.swap == 1, 'event_count_1'] = temp_event_count

    temp_event_per_sec = train.loc[train.swap == 1, 'event_per_sec_0']
    train.loc[train.swap == 1, 'event_per_sec_0'] = train.loc[train.swap == 1, 'event_per_sec_1']
    train.loc[train.swap == 1, 'event_per_sec_1'] = temp_event_per_sec

    temp_worker = train.loc[train.swap == 1, 'worker_0']
    train.loc[train.swap == 1, 'worker_0'] = train.loc[train.swap == 1, 'worker_1']
    train.loc[train.swap == 1, 'worker_1'] = temp_worker

    temp_worker_attack = train.loc[train.swap == 1, 'worker_attack_0']
    train.loc[train.swap == 1, 'worker_attack_0'] = train.loc[train.swap == 1, 'worker_attack_1']
    train.loc[train.swap == 1, 'worker_attack_1'] = temp_worker_attack

    temp_building_count = train.loc[train.swap == 1, 'building_count_0']
    train.loc[train.swap == 1, 'building_count_0'] = train.loc[train.swap == 1, 'building_count_1']
    train.loc[train.swap == 1, 'building_count_1'] = temp_building_count

    temp_unit_mineral = train.loc[train.swap == 1, 'building_mineral_0']
    train.loc[train.swap == 1, 'building_mineral_0'] = train.loc[train.swap == 1, 'building_mineral_1']
    train.loc[train.swap == 1, 'building_mineral_1'] = temp_unit_mineral

    temp_building_gas = train.loc[train.swap == 1, 'building_gas_0']
    train.loc[train.swap == 1, 'building_gas_0'] = train.loc[train.swap == 1, 'building_gas_1']
    train.loc[train.swap == 1, 'building_gas_1'] = temp_building_gas

    temp_target_worker = train.loc[train.swap == 1, 'target_worker_0']
    train.loc[train.swap == 1, 'target_worker_0'] = train.loc[train.swap == 1, 'target_worker_1']
    train.loc[train.swap == 1, 'target_worker_1'] = temp_target_worker

    temp_unit_count = train.loc[train.swap == 1, 'unit_count_0']
    train.loc[train.swap == 1, 'unit_count_0'] = train.loc[train.swap == 1, 'unit_count_1']
    train.loc[train.swap == 1, 'unit_count_1'] = temp_unit_count

    temp_unit_mineral = train.loc[train.swap == 1, 'unit_mineral_0']
    train.loc[train.swap == 1, 'unit_mineral_0'] = train.loc[train.swap == 1, 'unit_mineral_1']
    train.loc[train.swap == 1, 'unit_mineral_1'] = temp_unit_mineral

    temp_unit_gas = train.loc[train.swap == 1, 'unit_gas_0']
    train.loc[train.swap == 1, 'unit_gas_0'] = train.loc[train.swap == 1, 'unit_gas_1']
    train.loc[train.swap == 1, 'unit_gas_1'] = temp_unit_gas

    temp_sum_30sec = train.loc[train.swap == 1, '0_move_sum_30sec']
    train.loc[train.swap == 1, '0_move_sum_30sec'] = train.loc[train.swap == 1, '1_move_sum_30sec']
    train.loc[train.swap == 1, '1_move_sum_30sec'] = temp_sum_30sec

    for kind in ['unit_ground_count', 'unit_sky_count', 'unit_can_atk_ground_count', 'unit_can_atk_sky_count', 'unit_cannot_atk_ground_count', 'unit_cannot_atk_sky_count']:
        try:
            temp_kind1 = train.loc[train.swap == 1, kind + '_0']
            train.loc[train.swap == 1, kind + '_0'] = train.loc[train.swap == 1, kind + '_1']
            train.loc[train.swap == 1, kind + '_1'] = temp_kind1

            train.loc[train.swap == 1, kind + '_diff'] = (train.loc[train.swap == 1, kind + '_diff']) * -1
        except KeyError as e:
            print('key error!!! ======', e)

    for event in events:
        # 이벤트들
        temp_event = train.loc[train.swap == 1, event + '_count_0'] 
        train.loc[train.swap == 1, event + '_count_0']  = train.loc[train.swap == 1, event + '_count_1'] 
        train.loc[train.swap == 1, event + '_count_1']  = temp_event
        # diff 는 -1
        train.loc[train.swap == 1, event + '_diff'] = (train.loc[train.swap == 1, event + '_diff']) * -1

    for i in ['x', 'y']:
        for j in ['std', 'min', 'max', 'mean']:
            try:
                temp_camera_pos = train.loc[train.swap == 1, "camera_pos_" + i + "_" + j + "_player_0"] 
                train.loc[train.swap == 1, "camera_pos_" + i + "_" + j + "_player_0"] = train.loc[train.swap == 1, "camera_pos_" + i + "_" + j + "_player_1"] 
                train.loc[train.swap == 1, "camera_pos_" + i + "_" + j + "_player_1"]  = temp_camera_pos
            except KeyError as e:
                print('key error!!! ======', e)

    for func in ['sum','min','median','max']:
        try:
            temp_time = train.loc[train.swap == 1, '0_move_' + func ] 
            train.loc[train.swap == 1, '0_move_' + func ] = train.loc[train.swap == 1, '1_move_' + func ] 
            train.loc[train.swap == 1, '1_move_' + func ] = temp_time  
        except KeyError as e:
            print('key error!!! ======', e)

    for unit_code in unit_code_unique:
        try:
            temp_unit_code = train.loc[train.swap == 1, '0_' + unit_code]    
            train.loc[train.swap == 1, '0_' + unit_code]   = train.loc[train.swap == 1, '1_' + unit_code]  
            train.loc[train.swap == 1, '1_' + unit_code]  = temp_unit_code

            train.loc[train.swap == 1, unit_code + '_diff'] = (train.loc[train.swap == 1, unit_code + '_diff']) * -1
        except KeyError as e:
            print('key error!!! ======', e)

    for attack in attack_unique:
        try:
            temp_attack = train.loc[train.swap == 1, '0_Attack_' + attack]    
            train.loc[train.swap == 1, '0_Attack_' + attack]   = train.loc[train.swap == 1, '1_Attack_' + attack]  
            train.loc[train.swap == 1, '1_Attack_' + attack]  = temp_attack
        except KeyError as e:
            print('key error!!! ======', e)

    for target in target_unique:
        try:
            temp_target = train.loc[train.swap == 1, '0_Target_' + target]     
            train.loc[train.swap == 1, '0_Target_' + target] = train.loc[train.swap == 1, '1_Target_' + target] 
            train.loc[train.swap == 1, '1_Target_' + target]  = temp_target
      
        except KeyError as e:
            print('key error!!! ======', e)

    for selection in selection_unique:
        try:
            temp_selection = train.loc[train.swap == 1, '0_' + selection]     
            train.loc[train.swap == 1, '0_' + selection] = train.loc[train.swap == 1, '1_' + selection] 
            train.loc[train.swap == 1, '1_' + selection]  = temp_selection

            train.loc[train.swap == 1, selection + '_diff'] = (train.loc[train.swap == 1, selection + '_diff']) * -1
        except KeyError as e:
            print('key error!!! ======', e)

    train.loc[train.swap == 1, 'event_per_sec_diff'] = (train.loc[train.swap == 1, 'event_per_sec_diff']) * -1
    train.loc[train.swap == 1, 'worker_diff'] = (train.loc[train.swap == 1, 'worker_diff']) * -1
    train.loc[train.swap == 1, 'worker_attack_diff'] = (train.loc[train.swap == 1, 'worker_attack_diff']) * -1
    train.loc[train.swap == 1, 'building_count_diff'] = (train.loc[train.swap == 1, 'building_count_diff']) * -1

    # winner 반전
    if answer:
        train.loc[train.swap == 1, 'winner'] = train.loc[train.swap == 1, 'winner'].apply(lambda x: winner_swap(x))

    train.drop(labels='swap',axis=1, inplace=True)
    return swaped_index

In [ ]:
train['species'].unique()

array(['T', 'Z', 'P'], dtype=object)

In [ ]:
# Label Encoding

species_encoder = LabelEncoder()
species_encoder.fit(train['species'].unique())

#Encoding 결과 확인
# 0:프, 1:테, 2:저
species_encoder.classes_

array(['P', 'T', 'Z'], dtype=object)

In [ ]:
# event == Camera일 때 event_contents의 2차원 좌표 간 euclidean distance sum, min, median, max
def move_sum(i):
    return sum(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
def move_min(i):
    if len(i) == 1:
      return 0
    return min(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
def move_median(i):
    if len(i) == 1:
      return 0
    return np.median(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 + 
                             np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
def move_max(i):
    if len(i) == 1:
      return 0
    return max(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))

In [ ]:
def data_preparation(train, answer=False):
    df_train = pd.DataFrame(columns=['game_id'])
    df_train.game_id = train.game_id.unique()
    
    df_train['player_0'] = 0
    df_train['player_1'] = 1

    df_train['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time)
    df_train['time'] = (df_train.time*100//100*60 + df_train.time*100%100).astype(int) #시간을 초단위로 변경

    df = train[train.player==0]
    df_train['species_0']=np.array(df[df.shift(-1).game_id != df.game_id].species)
    df = train[train.player==1]
    df_train['species_1']=np.array(df[df.shift(-1).game_id != df.game_id].species)

    df_train['swap'] = df_train.apply(lambda row: need_swap(row), axis=1)

    df = train[train.player==0]
    df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
    df = train[train.player==1]
    df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])

    df_train['event_per_sec_0'], df_train['event_per_sec_1'] = df_train['event_count_0'] /df_train.time, df_train['event_count_1'] /df_train.time  # 이벤트당 걸린 시간

    for event in events:
        df = train[(train.player==0)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')
    
        df = train[(train.player==1)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')
    df_train = df_train.fillna(0)

    temp = train.groupby(['game_id','player'])[['camera_pos_x','camera_pos_y']].agg(['std','min','max','mean']).reset_index()
    temp.columns = ['game_id','player'] + [c[0]+'_'+c[1] for c in itertools.product(['camera_pos_x','camera_pos_y'],['std','min','max','mean'])]
    df_train = merge_data(temp, df_train)

    df_train.index = df_train['game_id']
    df_train = df_train.drop(labels='game_id',axis=1)

    contents = (train[train.event == 'Camera'].loc[:,['player','game_id','event_contents']].groupby(['player','game_id'])).agg([move_sum,move_min,move_median,move_max]).unstack(level=0)
    contents.columns = [y+x for x in ['sum','min','median','max'] for y in ['0_move_','1_move_']]
    for i in contents.columns:
        df_train[i] = contents[i].fillna(0)

    # 30초 이내 move_sum
    contents = (train[(train.time < (train.event == 'Camera'))].loc[:,['player','game_id','event_contents']].groupby(['player','game_id'])).agg(move_sum).unstack(level=0)
    contents.columns = ['0_move_sum_30sec','1_move_sum_30sec']
    for i in contents.columns:
        df_train[i] = contents[i]

    df_total_unit_code = []
    contents = pd.DataFrame(train.event_contents[(train.event == 'Ability')].map(lambda x: x[x.find('(')+1:x.find(')')]))  # event_contents의 16진수 코드만 추출
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    unit_code_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        if i in worker:
            df_total_unit_code.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        if i in total_unit_code:
            df_total_unit_code.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)

    worker_count_0 = 0
    worker_count_1 = 0
    for i in worker_0:
        try:
            worker_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in worker_1:
        try:
            worker_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['worker_0'] = worker_count_0
    df_train['worker_1'] = worker_count_1


    # event == Ability이고 event contents 가 Attack Target: '유닛' 일 때 
    contents = pd.DataFrame(train.event_contents[(train.event == 'Ability') & (train.event_contents.map(lambda x: str(x)[8:22]) == 'Attack; Target')].map(lambda x: x[x.find(':')+2:x.find(' [')]))  # Attack Target 만 추출
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    attack_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_Attack_','1_Attack_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_Attack_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)

    worker_attack_0 = 0
    worker_attack_1 = 0
    for i in attack_worker_0:
        try:
            worker_attack_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in attack_worker_1:
        try:
            worker_attack_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['worker_attack_0'] = worker_attack_0
    df_train['worker_attack_1'] = worker_attack_1


    unit_ground_count_0 = 0
    unit_ground_count_1 = 0
    for i in unit_ground_0:
        try:
            unit_ground_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_ground_1:
        try:
            unit_ground_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_ground_count_0'] = unit_ground_count_0
    df_train['unit_ground_count_1'] = unit_ground_count_1

    unit_sky_count_0 = 0
    unit_sky_count_1 = 0
    for i in unit_sky_0:
        try:
            unit_sky_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_sky_1:
        try:
            unit_sky_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_sky_count_0'] = unit_sky_count_0
    df_train['unit_sky_count_1'] = unit_sky_count_1

    unit_can_atk_ground_count_0 = 0
    unit_can_atk_ground_count_1 = 0
    for i in unit_can_atk_ground_0:
        try:
            unit_can_atk_ground_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_can_atk_ground_1:
        try:
            unit_can_atk_ground_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_can_atk_ground_count_0'] = unit_can_atk_ground_count_0
    df_train['unit_can_atk_ground_count_1'] = unit_can_atk_ground_count_1

    unit_can_atk_sky_count_0 = 0
    unit_can_atk_sky_count_1 = 0
    for i in unit_can_atk_sky_0:
        try:
            unit_can_atk_sky_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_can_atk_sky_1:
        try:
            unit_can_atk_sky_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_can_atk_sky_count_0'] = unit_can_atk_sky_count_0
    df_train['unit_can_atk_sky_count_1'] = unit_can_atk_sky_count_1

    unit_cannot_atk_ground_count_0 = 0
    unit_cannot_atk_ground_count_1 = 0
    for i in unit_cannot_atk_ground_0:
        try:
            unit_cannot_atk_ground_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_cannot_atk_ground_1:
        try:
            unit_cannot_atk_ground_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_cannot_atk_ground_count_0'] = unit_cannot_atk_ground_count_0
    df_train['unit_cannot_atk_ground_count_1'] = unit_cannot_atk_ground_count_1

    unit_cannot_atk_sky_count_0 = 0
    unit_cannot_atk_sky_count_1 = 0
    for i in unit_cannot_atk_sky_0: 
        try:
            unit_cannot_atk_sky_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in unit_cannot_atk_sky_1:
        try:
            unit_cannot_atk_sky_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['unit_cannot_atk_sky_count_0'] = unit_cannot_atk_sky_count_0
    df_train['unit_cannot_atk_sky_count_1'] = unit_cannot_atk_sky_count_1
    
    df_total_unit_target =[]
    contents = pd.DataFrame(train.event_contents[(train.event == 'Right Click') & (train.event_contents.map(lambda x: str(x)[:6]) == 'Target')].map(lambda x: x[x.find(':')+2:x.find(' [')]))  # event_contents의 Target만 추출
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    target_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_Target_','1_Target_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_Target_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        if i in target_worker:
            df_total_unit_target.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)
    
    target_worker_count_0 = 0
    target_worker_count_1 = 0
    for i in target_worker_0:
        try:
            target_worker_count_0 += df_train[i]
        except KeyError as e:
            print('key error!!! ======', e)
    for j in target_worker_1:
        try:
            target_worker_count_1 += df_train[j]
        except KeyError as e:
            print('key error!!! ======', e)

    df_train['target_worker_0'] = target_worker_count_0
    df_train['target_worker_1'] = target_worker_count_1

    # event == Selection일 때 event_contents 더미 변수 생성, 카운트
    contents = train[train.event == 'Selection'].event_contents.map(lambda x: re.sub('\s\[.....\]', '', re.sub('\s\[......\]', '', re.sub('\s\[.......\]', '', x))).
                                                            replace('[', '').replace(']', '').replace(' ', '').replace('\'', ''))
    contents = contents.str.split(',')
    max_num = max(contents.map(lambda x: len(x)))
    t = [0 for x in range(max_num)]
    for i in range(max_num):
        t[i] = pd.DataFrame(contents[contents.map(lambda x: len(x) > i)].map(lambda x: x[i]))

    contents = pd.concat([t[i] for i in range(max_num)])
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    selection_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)

    unit_count_0 = 0
    unit_count_1 = 0
    for j in unit_code_P_0 + unit_code_T_0 + unit_code_Z_0:
        try:
            unit_count_0 += df_train[j]
        except KeyError:    # 데이터에 해당 유닛 코드가 없을 때
            unit_count_0 += 0

    for j in unit_code_P_1 + unit_code_T_1 + unit_code_Z_1:
        try:
            unit_count_1 += df_train[j]
        except KeyError:
            unit_count_1 += 0

    df_train['unit_count_0'], df_train['unit_count_1'] = unit_count_0, unit_count_1

    building_count_0 = 0
    building_count_1 = 0
    for i in building_0:
        try:
            building_count_0 += df_train[i]
        except KeyError:    # 데이터에 해당 유닛 코드가 없을 때
            building_count_0 += 0


    for j in building_1:
        try:
            building_count_1 += df_train[j]
        except KeyError:
            building_count_1 += 0


    df_train['building_count_0'], df_train['building_count_1'] = building_count_0, building_count_1

    unit_mineral_0 = 0
    unit_mineral_1 = 0
    for i in unit_code_P_0 + unit_code_T_0 + unit_code_Z_0:
        try:
            unit_mineral_0 += df_train[i] * unit_mineral[i.split('_')[1]]
        except KeyError:
            unit_mineral_0 +=0

    for i in unit_code_P_1 + unit_code_T_1 + unit_code_Z_1:
        try:
            unit_mineral_1 += df_train[i] * unit_mineral[i.split('_')[1]]
        except KeyError:
            unit_mineral_1 +=0

    df_train['unit_mineral_0'] = unit_mineral_0
    df_train['unit_mineral_1'] = unit_mineral_1

    unit_gas_0 = 0
    unit_gas_1 = 0
    for i in unit_code_P_0 + unit_code_T_0 + unit_code_Z_0:
        try:
            unit_gas_0 += df_train[i] * unit_gas[i.split('_')[1]]
        except KeyError:
            unit_gas_0 +=0
    
    for i in unit_code_P_1 + unit_code_T_1 + unit_code_Z_1:
        try:
            unit_gas_1 += df_train[i] * unit_gas[i.split('_')[1]]
        except KeyError:
            unit_gas_1 +=0
    
    df_train['unit_gas_0'] = unit_gas_0
    df_train['unit_gas_1'] = unit_gas_1

    building_mineral_0 = 0
    building_mineral_1 = 0
    for i in building_0:
        try:
            building_mineral_0 += df_train[i] * building_mineral[i.split('_')[1]]
        except KeyError:
            building_mineral_0 +=0

    for i in building_1:
        try:
            building_mineral_1 += df_train[i] * building_mineral[i.split('_')[1]]
        except KeyError:
            building_mineral_1 +=0

    df_train['building_mineral_0'] = building_mineral_0
    df_train['building_mineral_1'] = building_mineral_1

    building_gas_0 = 0
    building_gas_1 = 0
    for i in building_0:
        try:
            building_gas_0 += df_train[i] * building_gas[i.split('_')[1]]
        except KeyError:
            building_gas_0 +=0
    
    for i in building_1:
        try:
            building_gas_1 += df_train[i] * building_gas[i.split('_')[1]]
        except KeyError:
            building_gas_1 +=0
    
    df_train['building_gas_0'] = building_gas_0
    df_train['building_gas_1'] = building_gas_1

    try:
        df_train.drop(labels=worker, axis=1, inplace=True)
    except KeyError as e:
        pass
    try:
        df_train.drop(labels=target_worker, axis=1, inplace=True)
    except KeyError as e:
        pass
    try:
        df_train.drop(labels=attack_worker, axis=1, inplace=True)
    except KeyError as e:
        pass
    

    # df_train.drop(labels=df_total_unit_code , axis=1, inplace=True)
    # df_train.drop(labels=df_total_unit_target , axis=1, inplace=True)

    for event in events:
        df_train[event + '_diff'] = df_train[event+'_count_0'] - df_train[event+'_count_1']

    df_train['event_per_sec_diff'] = df_train['event_per_sec_0'] - df_train['event_per_sec_1']

    df_train['worker_diff'] = df_train['worker_0'] - df_train['worker_1']
    df_train['building_count_diff'] = df_train['building_count_0'] - df_train['building_count_1']
    df_train['worker_attack_diff'] = df_train['worker_attack_0'] - df_train['worker_attack_1']

    for kind in ['unit_ground_count', 'unit_sky_count', 'unit_can_atk_ground_count', 'unit_can_atk_sky_count', 'unit_cannot_atk_ground_count', 'unit_cannot_atk_sky_count']:
        try:
            df_train[kind + '_diff'] = df_train[kind + '_0'] - df_train[kind + '_1']
        except KeyError as e:
          pass
    
    for unit_code in unit_code_unique:
        try:
            df_train[unit_code + '_diff'] = df_train['0_' + unit_code] - df_train['1_' + unit_code]
        except KeyError as e:
          pass

    for selection in selection_unique:
        try:
            df_train[selection + '_diff'] = df_train['0_' + selection] - df_train['1_' + selection]
        except KeyError as e:
            pass

    if answer:
        df_train['winner'] = np.array(train[train.shift(-1).game_id != train.game_id].winner)

    df_train['species_0'] = species_encoder.transform(df_train['species_0'])
    df_train['species_1'] = species_encoder.transform(df_train['species_1'])  

    swaped_index = swap(df_train, unit_code_unique, target_unique, selection_unique, attack_unique, answer=answer)
    df_train['species_war_kind'] = df_train['species_0'].astype(str) + '_' + df_train['species_1'].astype(str)
    df_train['species_war_kind'] = df_train['species_war_kind'].map({'0_0':0,'0_1':1,'0_2':2, '1_1':3,'1_2':4,'2_2':5})
    df_train.drop(labels=['player_0', 'player_1'], axis=1, inplace=True)
    
    if answer:
        x_data = df_train.drop(labels='winner', axis=1, inplace=False)
    else:
        x_data = df_train.iloc[:, :]
    y_data = pd.Series()

    x_data = x_data[sorted(x_data.columns)]

    if answer:
        y_data = df_train['winner']
    return x_data, y_data, swaped_index, unit_code_unique, target_unique, selection_unique, attack_unique

In [ ]:
x_train, y_train, _, unit_code_unique, target_unique, selection_unique, attack_unique = data_preparation(train, answer=True)

key error!!! ====== '0_1360'
key error!!! ====== '0_1820'
key error!!! ====== '0_15E0'
key error!!! ====== '0_Attack_Drone'
key error!!! ====== '0_Attack_SCV'
key error!!! ====== '0_Attack_Probe'
key error!!! ====== '0_Target_Probe'
key error!!! ====== '0_Target_SCV'
key error!!! ====== '0_Target_Drone'


In [ ]:
x_train.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,548,1.333333,1.234234,0.099099,1,1,3,0,0,444,3,7,-4,3,5,-2,0,0,0,0,2,-2,5,7,100,250,3,7,-4,325,550,1,0,1,9,6,0,0,0,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,655,2.600446,1.462054,1.138393,0,2,2,1,0,448,0,3,-3,2,3,-1,6,0,6,4,0,4,8,7,600,0,0,3,-3,1000,850,4,0,4,22,23,4,0,4,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,815,1.631579,3.063910,-1.432331,0,0,0,2,0,266,0,4,-4,0,3,-3,3,0,3,3,4,-1,3,4,75,250,0,3,-3,300,775,0,1,-1,7,8,0,0,0,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1155,2.063439,1.928214,0.135225,0,1,1,2,2,599,1,5,-4,0,5,-5,3,0,3,4,0,4,5,7,175,200,0,5,-5,900,450,2,2,0,16,9,2,0,2,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,393,1.616949,1.332203,0.284746,2,2,5,0,1,295,3,6,-3,0,0,0,0,0,0,3,6,-3,4,9,0,0,3,6,-3,250,600,0,0,0,6,6,0,0,0,0


In [ ]:
y_train

game_id
0        1
7        1
14       1
16       1
18       1
        ..
38865    1
38866    1
38867    0
38868    1
38871    1
Name: winner, Length: 19436, dtype: int64

In [ ]:
x_test, _, swaped_index, _, _, _, _ = data_preparation(test, answer=False)

key error!!! ====== '0_1840'
key error!!! ====== '1_1840'
key error!!! ====== '0_1840'
key error!!! ====== '1_1840'
key error!!! ====== '0_1840'
key error!!! ====== '1_1840'
key error!!! ====== '0_1840'
key error!!! ====== '1_1840'
key error!!! ====== '0_15E0'
key error!!! ====== '0_1820'
key error!!! ====== '0_1360'
key error!!! ====== '0_Attack_Probe'
key error!!! ====== '0_Attack_SCV'
key error!!! ====== '0_Attack_Drone'
key error!!! ====== '0_Target_Probe'
key error!!! ====== '0_Target_SCV'
key error!!! ====== '0_Target_Drone'


In [ ]:
x_test.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38872,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,715,1.340376,1.678404,-0.338028,0,0,0,1,0,426,2,4,-2,1,2,-1,0,0,0,2,3,-1,2,4,150,150,1,4,-3,525,375,1,0,1,11,12,1,0,1,-1
38873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,421,1.059946,1.147139,-0.087193,0,2,2,0,0,367,4,5,-1,2,3,-1,0,0,0,3,2,1,4,11,250,0,3,5,-2,675,1150,1,0,1,13,7,0,0,0,6
38874,1,0,0,3,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,824,1.669039,2.932384,-1.263345,1,1,3,0,1,281,2,1,1,0,1,-1,6,0,6,2,0,2,3,1,200,100,2,1,1,250,150,0,0,0,9,5,1,1,0,4
38875,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1223,1.805439,2.558577,-0.753138,0,1,1,1,5,478,2,7,-5,2,5,-3,0,3,-3,2,2,0,3,10,250,375,2,7,-5,400,775,1,1,0,19,21,1,5,-4,-2
38876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,213,3.423529,1.252941,2.170588,2,2,5,0,0,170,0,1,-1,0,0,0,0,0,0,0,1,-1,0,2,0,0,0,1,-1,0,150,0,0,0,4,2,0,0,0,2


In [ ]:
del train
del test

In [ ]:
swaped_index[:10]

array([38873, 38875, 38882, 38885, 38890, 38892, 38898, 38900, 38901,
       38906])

In [ ]:
x_train.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_attack_1  worker_attack_diff  worker_diff
game_id                      ...                                                  
0         5       0       1  ...                0                   0            3
7        13       0       0  ...                0                   4           -1
14        0       0       0  ...                0                   0           -1
16        4       0       0  ...                0                   2            7
18        6       0       0  ...                0                   0            0
...      ..     ...     ...  ...              ...                 ...          ...
38865     0       0       0  ...                0                   0            2
38866     0       0       0  ...                0                   0            0
38867     2       0       0  ...                0                   0           -5
38868     0       0       0  ...                6      

In [ ]:
y_train.head(10)

game_id
0     1
7     1
14    1
16    1
18    1
20    1
21    0
23    1
24    0
25    0
Name: winner, dtype: int64

In [ ]:
print(x_train.shape)
print(y_train.shape)

(19436, 2406)
(19436,)


In [ ]:
x_test.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38872,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,715,1.340376,1.678404,-0.338028,0,0,0,1,0,426,2,4,-2,1,2,-1,0,0,0,2,3,-1,2,4,150,150,1,4,-3,525,375,1,0,1,11,12,1,0,1,-1
38873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,421,1.059946,1.147139,-0.087193,0,2,2,0,0,367,4,5,-1,2,3,-1,0,0,0,3,2,1,4,11,250,0,3,5,-2,675,1150,1,0,1,13,7,0,0,0,6
38874,1,0,0,3,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,824,1.669039,2.932384,-1.263345,1,1,3,0,1,281,2,1,1,0,1,-1,6,0,6,2,0,2,3,1,200,100,2,1,1,250,150,0,0,0,9,5,1,1,0,4
38875,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1223,1.805439,2.558577,-0.753138,0,1,1,1,5,478,2,7,-5,2,5,-3,0,3,-3,2,2,0,3,10,250,375,2,7,-5,400,775,1,1,0,19,21,1,5,-4,-2
38876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,213,3.423529,1.252941,2.170588,2,2,5,0,0,170,0,1,-1,0,0,0,0,0,0,0,1,-1,0,2,0,0,0,1,-1,0,150,0,0,0,4,2,0,0,0,2


In [ ]:
x_test.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_attack_1  worker_attack_diff  worker_diff
game_id                      ...                                                  
38872     4       0       0  ...                0                   1           -1
38873     1       0       0  ...                0                   0            6
38874     1       0       0  ...                1                   0            4
38875    11       0       0  ...                5                  -4           -2
38876     0       0       0  ...                0                   0            2
...      ..     ...     ...  ...              ...                 ...          ...
55654     4       0       1  ...                0                   1           12
55655    11       0       1  ...                0                   6           -1
55656     6       0       0  ...                0                   0           -2
55657     0       0       0  ...                0      

In [ ]:
useless = []
for c in x_train.columns:
    try:
        if x_train[c].nunique()<2:
            useless.append(c)
            print("useless", c)
    except:
        print("exception:", c)

useless 0_11E0
useless 0_152C
useless 0_152F
useless 0_1780
useless 0_17C0
useless 0_1826
useless 0_182A
useless 0_182B
useless 0_1840
useless 0_1860
useless 0_1880
useless 0_18A0
useless 0_18C0
useless 0_1980
useless 0_1A63
useless 0_1AE0
useless 0_1B00
useless 0_1B23
useless 0_1B60
useless 0_1BE3
useless 0_2061
useless 0_2101
useless 0_2240
useless 0_2D80
useless 0_2DA0
useless 0_3161
useless 0_3FC0
useless 0_4041
useless 0_42A3
useless 0_5641
useless 0_960
useless 0_A80
useless 0_Attack_AiurLightBridgeAbandonedNE8Out
useless 0_Attack_Archon
useless 0_Attack_BroodLordWeapon
useless 0_Attack_RoboticsBay
useless 0_Attack_SlaynElemental
useless 0_Attack_TemplarArchive
useless 0_Baneling2
useless 0_BanelingCocoon2
useless 0_BanelingNest2
useless 0_BroodLord
useless 0_ChangelingMarineShield
useless 0_Corruptor
useless 0_DA0
useless 0_Disruptor
useless 0_EA2
useless 0_Egg2
useless 0_ExtendingBridgeNWWide8
useless 0_Infestor
useless 0_InfestorBurrowed
useless 0_Lurker
useless 0_SCV2
useless

In [ ]:
x_train.drop(labels=useless, axis=1, inplace=True)

In [ ]:
x_train

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,548,1.333333,1.234234,0.099099,1,1,3,0,0,444,3,7,-4,3,5,-2,0,0,0,0,2,-2,5,7,100,250,3,7,-4,325,550,1,0,1,9,6,0,0,0,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,655,2.600446,1.462054,1.138393,0,2,2,1,0,448,0,3,-3,2,3,-1,6,0,6,4,0,4,8,7,600,0,0,3,-3,1000,850,4,0,4,22,23,4,0,4,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,815,1.631579,3.063910,-1.432331,0,0,0,2,0,266,0,4,-4,0,3,-3,3,0,3,3,4,-1,3,4,75,250,0,3,-3,300,775,0,1,-1,7,8,0,0,0,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1155,2.063439,1.928214,0.135225,0,1,1,2,2,599,1,5,-4,0,5,-5,3,0,3,4,0,4,5,7,175,200,0,5,-5,900,450,2,2,0,16,9,2,0,2,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,393,1.616949,1.332203,0.284746,2,2,5,0,1,295,3,6,-3,0,0,0,0,0,0,3,6,-3,4,9,0,0,3,6,-3,250,600,0,0,0,6,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58,2.256410,1.487179,0.769231,0,2,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,2
38866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,79,1.702703,2.135135,-0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0
38867,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,831,0.910359,1.655378,-0.745020,0,1,1,1,0,502,4,3,1,2,3,-1,4,0,4,6,0,6,10,3,800,0,1,3,-2,1700,150,5,0,5,6,11,0,0,0,-5


In [ ]:
# x_train, x_test에만 있는 컬럼 삭제
x_train.drop(set(x_train.columns) - set(x_test.columns), axis=1, inplace=True)
x_test.drop(set(x_test.columns) - set(x_train.columns), axis=1, inplace=True)

In [ ]:
x_train.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_attack_1  worker_attack_diff  worker_diff
game_id                      ...                                                  
0         5       0       1  ...                0                   0            3
7        13       0       0  ...                0                   4           -1
14        0       0       0  ...                0                   0           -1
16        4       0       0  ...                0                   2            7
18        6       0       0  ...                0                   0            0
...      ..     ...     ...  ...              ...                 ...          ...
38865     0       0       0  ...                0                   0            2
38866     0       0       0  ...                0                   0            0
38867     2       0       0  ...                0                   0           -5
38868     0       0       0  ...                6      

In [ ]:
x_test.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_attack_1  worker_attack_diff  worker_diff
game_id                      ...                                                  
38872     4       0       0  ...                0                   1           -1
38873     1       0       0  ...                0                   0            6
38874     1       0       0  ...                1                   0            4
38875    11       0       0  ...                5                  -4           -2
38876     0       0       0  ...                0                   0            2
...      ..     ...     ...  ...              ...                 ...          ...
55654     4       0       1  ...                0                   1           12
55655    11       0       1  ...                0                   6           -1
55656     6       0       0  ...                0                   0           -2
55657     0       0       0  ...                0      

In [ ]:
y_train

game_id
0        1
7        1
14       1
16       1
18       1
        ..
38865    1
38866    1
38867    0
38868    1
38871    1
Name: winner, Length: 19436, dtype: int64

In [ ]:
x_train_copy = x_train.copy()

In [ ]:
x_train_copy

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,548,1.333333,1.234234,0.099099,1,1,3,0,0,444,3,7,-4,3,5,-2,0,0,0,0,2,-2,5,7,100,250,3,7,-4,325,550,1,0,1,9,6,0,0,0,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,655,2.600446,1.462054,1.138393,0,2,2,1,0,448,0,3,-3,2,3,-1,6,0,6,4,0,4,8,7,600,0,0,3,-3,1000,850,4,0,4,22,23,4,0,4,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,815,1.631579,3.063910,-1.432331,0,0,0,2,0,266,0,4,-4,0,3,-3,3,0,3,3,4,-1,3,4,75,250,0,3,-3,300,775,0,1,-1,7,8,0,0,0,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1155,2.063439,1.928214,0.135225,0,1,1,2,2,599,1,5,-4,0,5,-5,3,0,3,4,0,4,5,7,175,200,0,5,-5,900,450,2,2,0,16,9,2,0,2,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,393,1.616949,1.332203,0.284746,2,2,5,0,1,295,3,6,-3,0,0,0,0,0,0,3,6,-3,4,9,0,0,3,6,-3,250,600,0,0,0,6,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58,2.256410,1.487179,0.769231,0,2,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,2
38866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,79,1.702703,2.135135,-0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0
38867,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,831,0.910359,1.655378,-0.745020,0,1,1,1,0,502,4,3,1,2,3,-1,4,0,4,6,0,6,10,3,800,0,1,3,-2,1700,150,5,0,5,6,11,0,0,0,-5


In [ ]:
y_train_copy = y_train.copy()

In [ ]:
y_train_copy

game_id
0        1
7        1
14       1
16       1
18       1
        ..
38865    1
38866    1
38867    0
38868    1
38871    1
Name: winner, Length: 19436, dtype: int64

In [ ]:
x_train_copy['swap'] = 1

In [ ]:
x_train_copy

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff,swap
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,1.333333,1.234234,0.099099,1,1,3,0,0,444,3,7,-4,3,5,-2,0,0,0,0,2,-2,5,7,100,250,3,7,-4,325,550,1,0,1,9,6,0,0,0,3,1
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.600446,1.462054,1.138393,0,2,2,1,0,448,0,3,-3,2,3,-1,6,0,6,4,0,4,8,7,600,0,0,3,-3,1000,850,4,0,4,22,23,4,0,4,-1,1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.631579,3.063910,-1.432331,0,0,0,2,0,266,0,4,-4,0,3,-3,3,0,3,3,4,-1,3,4,75,250,0,3,-3,300,775,0,1,-1,7,8,0,0,0,-1,1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.063439,1.928214,0.135225,0,1,1,2,2,599,1,5,-4,0,5,-5,3,0,3,4,0,4,5,7,175,200,0,5,-5,900,450,2,2,0,16,9,2,0,2,7,1
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.616949,1.332203,0.284746,2,2,5,0,1,295,3,6,-3,0,0,0,0,0,0,3,6,-3,4,9,0,0,3,6,-3,250,600,0,0,0,6,6,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.256410,1.487179,0.769231,0,2,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,2,1
38866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.702703,2.135135,-0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,1
38867,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.910359,1.655378,-0.745020,0,1,1,1,0,502,4,3,1,2,3,-1,4,0,4,6,0,6,10,3,800,0,1,3,-2,1700,150,5,0,5,6,11,0,0,0,-5,1


In [ ]:
_ = swap(x_train_copy, unit_code_unique, target_unique, selection_unique, attack_unique, answer=False)

key error!!! ====== '0_1360'
key error!!! ====== '0_1820'
key error!!! ====== '0_15E0'
key error!!! ====== '1_1583'
key error!!! ====== '0_182B'
key error!!! ====== '0_1780'
key error!!! ====== '0_1840'
key error!!! ====== '1_3180'
key error!!! ====== '0_182A'
key error!!! ====== '0_5641'
key error!!! ====== '0_1B23'
key error!!! ====== '1_1684'
key error!!! ====== '0_EA2'
key error!!! ====== '0_1826'
key error!!! ====== '1_B61'
key error!!! ====== '1_16A1'
key error!!! ====== '0_1980'
key error!!! ====== '0_2D80'
key error!!! ====== '0_3FC0'
key error!!! ====== '0_1860'
key error!!! ====== '0_A80'
key error!!! ====== '0_2101'
key error!!! ====== '0_1880'
key error!!! ====== '0_42A3'
key error!!! ====== '1_1660'
key error!!! ====== '0_1A63'
key error!!! ====== '0_2061'
key error!!! ====== '0_152C'
key error!!! ====== '0_17C0'
key error!!! ====== '0_1B60'
key error!!! ====== '1_4AFE'
key error!!! ====== '0_1AE0'
key error!!! ====== '0_1B00'
key error!!! ====== '0_11E0'
key error!!! ====

In [ ]:
x_train_copy

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4,1,0,3,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,592,1.234234,1.333333,-0.099099,1,1,3,0,0,444,7,3,4,5,3,2,0,0,0,2,0,2,7,5,250,100,7,3,4,550,325,0,1,-1,6,9,0,0,0,-3
7,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1165,1.462054,2.600446,-1.138393,2,0,2,0,1,448,3,0,3,3,2,1,0,6,-6,0,4,-4,7,8,0,600,3,0,3,850,1000,0,4,-4,23,22,0,4,-4,1
14,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,434,3.063910,1.631579,1.432331,0,0,0,0,2,266,4,0,4,3,0,3,0,3,-3,4,3,1,4,3,250,75,3,0,3,775,300,1,0,1,8,7,0,0,0,1
16,5,0,2,6,0,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,...,1236,1.928214,2.063439,-0.135225,1,0,1,2,2,599,5,1,4,5,0,5,0,3,-3,0,4,-4,7,5,200,175,5,0,5,450,900,2,2,0,9,16,0,2,-2,-7
18,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,477,1.332203,1.616949,-0.284746,2,2,5,1,0,295,6,3,3,0,0,0,0,0,0,6,3,3,9,4,0,0,6,3,3,600,250,0,0,0,6,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,1.487179,2.256410,-0.769231,2,0,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,-2
38866,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,63,2.135135,1.702703,0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0
38867,3,0,1,2,4,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,457,1.655378,0.910359,0.745020,1,0,1,0,1,502,3,4,-1,3,2,1,0,4,-4,0,6,-6,3,10,0,800,3,1,2,150,1700,0,5,-5,11,6,0,0,0,5


In [ ]:
y_train_copy = y_train_copy.apply(lambda x: winner_swap(x))

In [ ]:
y_train_copy

game_id
0        0
7        0
14       0
16       0
18       0
        ..
38865    0
38866    0
38867    1
38868    0
38871    0
Name: winner, Length: 19436, dtype: int64

In [ ]:
n = x_train.shape[0]
game_id_max = x_train.index.max() + 1

In [ ]:
x_train_copy.index = [game_id_max + x for x in range(n)]
y_train_copy.index = [game_id_max + x for x in range(n)]

In [ ]:
x_train_copy

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
38872,4,1,0,3,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,592,1.234234,1.333333,-0.099099,1,1,3,0,0,444,7,3,4,5,3,2,0,0,0,2,0,2,7,5,250,100,7,3,4,550,325,0,1,-1,6,9,0,0,0,-3
38873,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1165,1.462054,2.600446,-1.138393,2,0,2,0,1,448,3,0,3,3,2,1,0,6,-6,0,4,-4,7,8,0,600,3,0,3,850,1000,0,4,-4,23,22,0,4,-4,1
38874,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,434,3.063910,1.631579,1.432331,0,0,0,0,2,266,4,0,4,3,0,3,0,3,-3,4,3,1,4,3,250,75,3,0,3,775,300,1,0,1,8,7,0,0,0,1
38875,5,0,2,6,0,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,...,1236,1.928214,2.063439,-0.135225,1,0,1,2,2,599,5,1,4,5,0,5,0,3,-3,0,4,-4,7,5,200,175,5,0,5,450,900,2,2,0,9,16,0,2,-2,-7
38876,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,477,1.332203,1.616949,-0.284746,2,2,5,1,0,295,6,3,3,0,0,0,0,0,0,6,3,3,9,4,0,0,6,3,3,600,250,0,0,0,6,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,1.487179,2.256410,-0.769231,2,0,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,-2
58304,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,63,2.135135,1.702703,0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0
58305,3,0,1,2,4,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,457,1.655378,0.910359,0.745020,1,0,1,0,1,502,3,4,-1,3,2,1,0,4,-4,0,6,-6,3,10,0,800,3,1,2,150,1700,0,5,-5,11,6,0,0,0,5
58306,0,0,1,3,2,1,1,4,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,776,1.345576,1.295492,0.050083,1,0,1,3,1,599,15,3,12,9,1,8,15,0,15,6,3,3,15,3,425,150,15,2,13,900,625,0,1,-1,7,11,6,0,6,-4


In [ ]:
y_train_copy

38872    0
38873    0
38874    0
38875    0
38876    0
        ..
58303    0
58304    0
58305    1
58306    0
58307    0
Name: winner, Length: 19436, dtype: int64

In [ ]:
x_train = pd.concat([x_train, x_train_copy])
y_train = pd.concat([y_train, y_train_copy])

In [ ]:
x_train

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,species_war_kind,target_worker_0,target_worker_1,time,unit_can_atk_ground_count_0,unit_can_atk_ground_count_1,unit_can_atk_ground_count_diff,unit_can_atk_sky_count_0,unit_can_atk_sky_count_1,unit_can_atk_sky_count_diff,unit_cannot_atk_ground_count_0,unit_cannot_atk_ground_count_1,unit_cannot_atk_ground_count_diff,unit_cannot_atk_sky_count_0,unit_cannot_atk_sky_count_1,unit_cannot_atk_sky_count_diff,unit_count_0,unit_count_1,unit_gas_0,unit_gas_1,unit_ground_count_0,unit_ground_count_1,unit_ground_count_diff,unit_mineral_0,unit_mineral_1,unit_sky_count_0,unit_sky_count_1,unit_sky_count_diff,worker_0,worker_1,worker_attack_0,worker_attack_1,worker_attack_diff,worker_diff
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,548,1.333333,1.234234,0.099099,1,1,3,0,0,444,3,7,-4,3,5,-2,0,0,0,0,2,-2,5,7,100,250,3,7,-4,325,550,1,0,1,9,6,0,0,0,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,655,2.600446,1.462054,1.138393,0,2,2,1,0,448,0,3,-3,2,3,-1,6,0,6,4,0,4,8,7,600,0,0,3,-3,1000,850,4,0,4,22,23,4,0,4,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,815,1.631579,3.063910,-1.432331,0,0,0,2,0,266,0,4,-4,0,3,-3,3,0,3,3,4,-1,3,4,75,250,0,3,-3,300,775,0,1,-1,7,8,0,0,0,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1155,2.063439,1.928214,0.135225,0,1,1,2,2,599,1,5,-4,0,5,-5,3,0,3,4,0,4,5,7,175,200,0,5,-5,900,450,2,2,0,16,9,2,0,2,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,393,1.616949,1.332203,0.284746,2,2,5,0,1,295,3,6,-3,0,0,0,0,0,0,3,6,-3,4,9,0,0,3,6,-3,250,600,0,0,0,6,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,1.487179,2.256410,-0.769231,2,0,2,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,-2
58304,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,63,2.135135,1.702703,0.432432,1,1,3,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0
58305,3,0,1,2,4,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,457,1.655378,0.910359,0.745020,1,0,1,0,1,502,3,4,-1,3,2,1,0,4,-4,0,6,-6,3,10,0,800,3,1,2,150,1700,0,5,-5,11,6,0,0,0,5
58306,0,0,1,3,2,1,1,4,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,776,1.345576,1.295492,0.050083,1,0,1,3,1,599,15,3,12,9,1,8,15,0,15,6,3,3,15,3,425,150,15,2,13,900,625,0,1,-1,7,11,6,0,6,-4


In [ ]:
y_train

0        1
7        1
14       1
16       1
18       1
        ..
58303    0
58304    0
58305    1
58306    0
58307    0
Name: winner, Length: 38872, dtype: int64

In [ ]:
def cat_cv(learning_rate, n_estimators, subsample, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = CatBoostClassifier(
            # eval_metric = 'AUC',  
            # task_type = 'GPU',   
            # grow_policy = 'Depthwise', 
            # num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            # colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            # reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed1 = partial(cat_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
catBO = BayesianOptimization(
    func_fixed1, 
    {
        # 'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        # 'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        # 'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (20, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
catBO.maximize(init_points=5, n_iter=10) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
672:	learn: 0.6428298	total: 44.8s	remaining: 20.7s
673:	learn: 0.6428204	total: 44.9s	remaining: 20.6s
674:	learn: 0.6427973	total: 45s	remaining: 20.6s
675:	learn: 0.6427709	total: 45s	remaining: 20.5s
676:	learn: 0.6427357	total: 45.1s	remaining: 20.4s
677:	learn: 0.6427141	total: 45.1s	remaining: 20.4s
678:	learn: 0.6426806	total: 45.2s	remaining: 20.3s
679:	learn: 0.6426472	total: 45.3s	remaining: 20.2s
680:	learn: 0.6426180	total: 45.3s	remaining: 20.2s
681:	learn: 0.6425870	total: 45.4s	remaining: 20.1s
682:	learn: 0.6425552	total: 45.5s	remaining: 20s
683:	learn: 0.6425308	total: 45.5s	remaining: 20s
684:	learn: 0.6425128	total: 45.6s	remaining: 19.9s
685:	learn: 0.6424826	total: 45.7s	remaining: 19.8s
686:	learn: 0.6424513	total: 45.7s	remaining: 19.8s
687:	learn: 0.6424448	total: 45.8s	remaining: 19.7s
688:	learn: 0.6424279	total: 45.9s	remaining: 19.6s
689:	learn: 0.6423938	total: 45.9s	remaining: 19.6s
690:	learn: 0.6423729	total: 46s	rem

In [ ]:
params1 = catBO.max['params']

In [ ]:
params1

{'learning_rate': 0.04566388076866954,
 'n_estimators': 935.2365313830772,
 'reg_lambda': 20.148605796209903,
 'subsample': 0.9216697676688711}

In [ ]:
models1 = cat_cv(
    # params1['num_leaves'], 
    params1['learning_rate'], 
    params1['n_estimators'], 
    params1['subsample'], 
    # params1['colsample_bytree'], 
    # params1['reg_alpha'], 
    params1['reg_lambda'], 
    x_data=x_train, y_data=y_train, n_splits=5, output='model')

0:	learn: 0.6900196	total: 113ms	remaining: 1m 45s
1:	learn: 0.6871194	total: 227ms	remaining: 1m 45s
2:	learn: 0.6844736	total: 332ms	remaining: 1m 43s
3:	learn: 0.6820495	total: 432ms	remaining: 1m 40s
4:	learn: 0.6796216	total: 545ms	remaining: 1m 41s
5:	learn: 0.6774293	total: 651ms	remaining: 1m 40s
6:	learn: 0.6753709	total: 777ms	remaining: 1m 42s
7:	learn: 0.6736353	total: 881ms	remaining: 1m 42s
8:	learn: 0.6718204	total: 990ms	remaining: 1m 41s
9:	learn: 0.6702412	total: 1.12s	remaining: 1m 43s
10:	learn: 0.6690166	total: 1.22s	remaining: 1m 42s
11:	learn: 0.6676366	total: 1.33s	remaining: 1m 42s
12:	learn: 0.6662272	total: 1.45s	remaining: 1m 42s
13:	learn: 0.6650381	total: 1.56s	remaining: 1m 42s
14:	learn: 0.6640765	total: 1.69s	remaining: 1m 43s
15:	learn: 0.6629009	total: 1.8s	remaining: 1m 43s
16:	learn: 0.6619981	total: 1.91s	remaining: 1m 42s
17:	learn: 0.6609296	total: 2.02s	remaining: 1m 42s
18:	learn: 0.6599371	total: 2.15s	remaining: 1m 43s
19:	learn: 0.6589918	to

In [ ]:
preds1 = []
for model in models1:
    pred = model.predict_proba(x_test)[:, 1]
    preds1.append(pred)
pred1 = np.mean(preds1, axis=0)

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed2 = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed2, 
    {
        'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (20, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=10) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7404   |  0.0708   |  0.08152  |  790.0    |  304.6    |  1.931    |  49.37    |  0.4062   |
|  2        |  0.7208   |  0.7578   |  0.009006 |  328.4    |  639.9    |  4.599    |  26.55    |  0.6635   |
|  3        |  0.7366   |  0.6787   |  0.09504  |  299.5    |  640.8    |  3.833    |  32.01    |  0.9427   |
|  4        |  0.7336   |  0.9299   |  0.09484  |  394.5    |  361.0    |  6.648    |  21.27    |  0.2322   |
|  5        |  0.7365   |  0.4301   |  0.007886 |  788.7    |  876.7    |  1.504    |  23.03    |  0.2709   |
|  6        |  0.6746   |  0.6222   |  0.03011  |  17.29    |  17.06    |  8.005    |  38.34    |  0.4728   |
|  7        |  0.6885   |  0.9608   |  0.02043  |  27.04    |  1.023e+0 |  3.621    |  23.66    |  0.5903   |
|  8      

In [ ]:
params2 = lgbBO.max['params']

In [ ]:
params2

{'colsample_bytree': 0.06752766992829817,
 'learning_rate': 0.0730810067264372,
 'n_estimators': 1023.1495861088881,
 'num_leaves': 1023.2406977725142,
 'reg_alpha': 5.500707757115696,
 'reg_lambda': 45.86574388586635,
 'subsample': 0.9920047464177493}

In [ ]:
models2 = lgb_cv(
    params2['num_leaves'], 
    params2['learning_rate'], 
    params2['n_estimators'], 
    params2['subsample'], 
    params2['colsample_bytree'], 
    params2['reg_alpha'], 
    params2['reg_lambda'], 
    x_data=x_train, y_data=y_train, n_splits=5, output='model')

In [ ]:
preds2 = []
for model in models2:
    pred = model.predict_proba(x_test)[:, 1]
    preds2.append(pred)
pred2 = np.mean(preds2, axis=0)

In [ ]:
submission1 = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/sample_submission.csv', index_col=0)
submission2 = submission1.copy()

In [ ]:
submission1['winner'] = submission1['winner'] + pred1 # pred는 pred1 과 pred2의 비율

In [ ]:
submission1.head()

,winner
game_id,
38872,0.568804
38873,0.426426
38874,0.470966
38875,0.791357
38876,0.634816


In [ ]:
submission2['winner'] = submission2['winner'] + pred2

In [ ]:
submission2.head()

,winner
game_id,
38872,0.527465
38873,0.372201
38874,0.380122
38875,0.965494
38876,0.710580


In [ ]:
for i in swaped_index:
    submission1.loc[i, :] = 1 - submission1.loc[i, :]
    submission2.loc[i, :] = 1 - submission2.loc[i, :]

In [ ]:
submission1.head()

,winner
game_id,
38872,0.568804
38873,0.573574
38874,0.470966
38875,0.208643
38876,0.634816


In [ ]:
submission2.head()

,winner
game_id,
38872,0.527465
38873,0.627799
38874,0.380122
38875,0.034506
38876,0.710580


In [ ]:
submission1['winner'] = submission1['winner']*0.6 + submission2['winner']*0.4

In [ ]:
submission1.head()

,winner
game_id,
38872,0.552269
38873,0.595264
38874,0.434629
38875,0.138988
38876,0.665122


In [ ]:
submission1.to_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/results/submission_master_attack.csv')